In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

In [49]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

In [4]:
telcom = pd.read_csv(r"WA_Fn-UseC_-Telco-Customer-Churn.csv")
#first few rows
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
telcom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [6]:
print ("Rows     : " ,telcom.shape[0])
print ("Columns  : " ,telcom.shape[1])
print ("\nFeatures : \n" ,telcom.columns.tolist())
print ("\nMissing values :  ", telcom.isnull().sum().values.sum())
print ("\nUnique values :  \n",telcom.nunique())

Rows     :  7043
Columns  :  21

Features : 
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Missing values :   0

Unique values :  
 customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64


In [7]:
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)

#Dropping null values from total charges column which contain .15% missing data 
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

In [8]:
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)

replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']

In [9]:
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'No internet service' : 'No'})

In [10]:
telcom["SeniorCitizen"] = telcom["SeniorCitizen"].replace({1:"Yes",0:"No"})

In [11]:
def tenure_lab(telcom) :    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"

In [12]:
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom), axis = 1)

churn     = telcom[telcom["Churn"] == "Yes"]
not_churn = telcom[telcom["Churn"] == "No"]

In [13]:
#Data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [14]:
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]
bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
multi_cols = [i for i in cat_cols if i not in bin_cols]

In [15]:
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,7590-VHVEG,Female,No,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,Tenure_0-12
1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,Tenure_24-48
2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Tenure_0-12
3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Tenure_24-48
4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Tenure_0-12


In [16]:
le = LabelEncoder()
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])

In [17]:
telcom = pd.get_dummies(data = telcom,columns = multi_cols )
telcom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 35 columns):
customerID                                 7032 non-null object
gender                                     7032 non-null int64
SeniorCitizen                              7032 non-null int64
Partner                                    7032 non-null int64
Dependents                                 7032 non-null int64
tenure                                     7032 non-null int64
PhoneService                               7032 non-null int64
OnlineSecurity                             7032 non-null int64
OnlineBackup                               7032 non-null int64
DeviceProtection                           7032 non-null int64
TechSupport                                7032 non-null int64
StreamingTV                                7032 non-null int64
StreamingMovies                            7032 non-null int64
PaperlessBilling                           7032 non-null int64
MonthlyCha

In [18]:
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

In [25]:
type(telcom)

pandas.core.frame.DataFrame

In [26]:
df_telcom_og = telcom.copy()
telcom = telcom.drop(num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
# from yellowbrick.classifier import DiscriminationThreshold
from sklearn.metrics import accuracy_score

In [27]:
#splitting train and test data 
train,test = train_test_split(telcom,test_size = .25 ,random_state = 111)

In [28]:
##seperating dependent and independent variables
cols    = [i for i in telcom.columns if i not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

In [142]:
len(cols), cols

(33,
 ['gender',
  'SeniorCitizen',
  'Partner',
  'Dependents',
  'PhoneService',
  'OnlineSecurity',
  'OnlineBackup',
  'DeviceProtection',
  'TechSupport',
  'StreamingTV',
  'StreamingMovies',
  'PaperlessBilling',
  'MultipleLines_No',
  'MultipleLines_No phone service',
  'MultipleLines_Yes',
  'InternetService_DSL',
  'InternetService_Fiber optic',
  'InternetService_No',
  'Contract_Month-to-month',
  'Contract_One year',
  'Contract_Two year',
  'PaymentMethod_Bank transfer (automatic)',
  'PaymentMethod_Credit card (automatic)',
  'PaymentMethod_Electronic check',
  'PaymentMethod_Mailed check',
  'tenure_group_Tenure_0-12',
  'tenure_group_Tenure_12-24',
  'tenure_group_Tenure_24-48',
  'tenure_group_Tenure_48-60',
  'tenure_group_Tenure_gt_60',
  'tenure',
  'MonthlyCharges',
  'TotalCharges'])

In [29]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [31]:
#select columns
cols = [i for i in telcom.columns if i not in Id_col + target_col ]

In [32]:
#dataframe with non negative values
df_x = df_telcom_og[cols]
df_y = df_telcom_og[target_col]

In [36]:
df_x.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges
0,0,0,1,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,1,29.85,29.85
1,1,0,0,0,1,1,0,1,0,0,...,0,1,0,0,1,0,0,34,56.95,1889.50
2,1,0,0,0,1,1,1,0,0,0,...,0,1,1,0,0,0,0,2,53.85,108.15
3,1,0,0,0,0,1,0,1,1,0,...,0,0,0,0,1,0,0,45,42.30,1840.75
4,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,2,70.70,151.65


In [37]:
df_y.head()

,Churn
0,0
1,0
2,1
3,0
4,1


In [33]:
#fit model with k= 3
select = SelectKBest(score_func = chi2,k = 3)
fit    = select.fit(df_x,df_y)

In [34]:
#Summerize scores
print ("scores")
print (fit.scores_)
print ("P - Values")
print (fit.pvalues_)

scores
[2.54297062e-01 1.33482766e+02 8.18577694e+01 1.31271509e+02
 9.29483891e-02 1.47165601e+02 3.12098318e+01 2.02160070e+01
 1.35439602e+02 1.73206148e+01 1.59306111e+01 1.04979224e+02
 3.88864216e+00 8.68247305e-01 6.51465136e+00 7.11376111e+01
 3.72082851e+02 2.85475152e+02 5.16714004e+02 1.76608724e+02
 4.86223101e+02 7.66190658e+01 9.99725387e+01 4.24113152e+02
 4.47251434e+01 4.96199796e+02 2.38616006e+00 3.12932969e+01
 6.29955193e+01 2.87501128e+02 1.63773281e+04 3.65307468e+03
 6.29630810e+05]
P - Values
[6.14065505e-001 7.08954608e-031 1.46240915e-019 2.15953960e-030
 7.60461827e-001 7.21988253e-034 2.31590182e-008 6.91717063e-006
 2.64595220e-031 3.15742928e-005 6.57073922e-005 1.23423173e-024
 4.86137123e-002 3.51440986e-001 1.06989295e-002 3.33158163e-017
 6.58713045e-083 4.81399951e-064 2.19511926e-114 2.66631661e-040
 9.45428638e-108 2.07328356e-018 1.54524820e-023 3.10584857e-094
 2.26727030e-011 6.38005232e-110 1.22413991e-001 2.21844282e-008
 2.07177341e-015 1.741

In [38]:
#create dataframe
score = pd.DataFrame({"features":cols,"scores":fit.scores_,"p_values":fit.pvalues_ })
score = score.sort_values(by = "scores" ,ascending =False)

In [39]:
#creating new label for categorical and numerical columns
score["feature_type"] = np.where(score["features"].isin(num_cols),"Numerical","Categorical")

In [41]:
#Decision Tree Visualization
#Using top three numerical features

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn import tree
from graphviz import Source
from IPython.display import SVG,display


In [42]:
#top 3 categorical features
features_cat  = score[score["feature_type"] == "Categorical"]["features"][:3].tolist()

#top 3 numerical features
features_num  = score[score["feature_type"] == "Numerical"]["features"][:3].tolist()


In [43]:
features_cat, features_num

(['Contract_Month-to-month', 'tenure_group_Tenure_0-12', 'Contract_Two year'],
 ['TotalCharges', 'tenure', 'MonthlyCharges'])

In [44]:
from sklearn.ensemble import RandomForestClassifier
   
def model_report(model,training_x,testing_x,training_y,testing_y,name) :
    model.fit(training_x,training_y)
    predictions  = model.predict(testing_x)
    accuracy     = accuracy_score(testing_y,predictions)
    
    df = pd.DataFrame({"Model"           : [name],
                       "Accuracy_score"  : [accuracy]
                      })
    return df


C:\Users\rahsood\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [45]:
rfc = RandomForestClassifier(random_state = 123)
model1 = model_report(rfc, train_X, test_X, train_Y, test_Y, "Random Forest Classifier")


In [50]:
model_performances = pd.concat([model1],axis = 0).reset_index()

model_performances = model_performances.drop("index",axis =1)

table  = ff.create_table(np.round(model_performances,4))

py.plot(table)


'temp-plot.html'

In [104]:
#Parameter tuning
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth': [7,9],
               'criterion': ['entropy','gini'],
#                'bootstrap': ['True','False'],
              'bootstrap': ['True'],
               'oob_score': ['True','False'],
               'max_features': ['auto','log2'],
               'n_estimators': [100,500,900]}


In [105]:
grid_search = GridSearchCV(estimator = rfc,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [106]:
train_X.values

array([[ 1.        ,  0.        ,  1.        , ..., -0.13941712,
         0.69810887,  0.06924418],
       [ 0.        ,  0.        ,  0.        , ...,  1.32736548,
         0.83273267,  1.57122278],
       [ 0.        ,  1.        ,  0.        , ..., -1.03578427,
         0.00504563, -0.78957134],
       ...,
       [ 1.        ,  0.        ,  1.        , ...,  1.53108529,
         0.48038397,  1.52145684],
       [ 1.        ,  1.        ,  1.        , ...,  0.91992587,
        -0.15949457,  0.45565822],
       [ 0.        ,  0.        ,  1.        , ..., -0.30239297,
         0.20781233, -0.19659333]])

In [107]:
train_Y.values

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [108]:
np.shape(train_X.values)

(5274, 33)

In [109]:
np.shape(train_X.values[:,0])

(5274,)

In [ ]:
# This step results in "too many indices for array" error
grid_search_results = grid_search.fit(train_X.values, train_Y.values)

In [110]:
grid_search_results = grid_search.fit(train_X.values, train_Y.values[:,0])

In [111]:
# grid_search_results = grid_search.fit(train_X, train_Y)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_


In [112]:
best_accuracy, best_parameters, best_estimator

(0.8028062191884717,
 {'bootstrap': 'True',
  'criterion': 'gini',
  'max_depth': 9,
  'max_features': 'auto',
  'n_estimators': 100,
  'oob_score': 'True'},
 RandomForestClassifier(bootstrap='True', class_weight=None, criterion='gini',
             max_depth=9, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score='True', random_state=123,
             verbose=0, warm_start=False))

In [66]:
len(train_X), len(train_Y)

(5274, 5274)

In [116]:
# rfc = RandomForestClassifier(bootstrap='True', class_weight=None, criterion='gini',
#             max_depth=7, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
#             oob_score='True', random_state=123, verbose=0,
#             warm_start=False)

In [114]:
rfc = RandomForestClassifier(bootstrap='True', class_weight=None, criterion='gini',
             max_depth=9, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score='True', random_state=123,
             verbose=0, warm_start=False)

In [115]:
model2 = model_report(rfc,train_X,test_X,train_Y,test_Y, "Random Forest Classifier After Parameter Tuning")

model_performances = pd.concat([model1,model2],axis = 0).reset_index()

model_performances = model_performances.drop("index",axis =1)

table  = ff.create_table(np.round(model_performances,4))

py.plot(table)


'temp-plot.html'

In [154]:
rfc.fit(train_X,train_Y)
pred_y  = rfc.predict(test_X)
# accuracy     = accuracy_score(train_X,pred_y)

In [155]:
np.shape(test_X)

(1758, 33)

In [147]:
test_Y

,Churn
3464,0
354,1
2326,0
2921,0
5391,0
1413,0
1781,1
1141,0
4004,0
3841,0


In [162]:
type(test_X), type(pred_y), test_X.shape, np.shape(pred_y)

(pandas.core.frame.DataFrame, numpy.ndarray, (1758, 33), (1758,))

In [163]:
test_X

,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges
3464,1,0,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,-0.995040,-0.448686,-0.825771
354,0,1,0,0,0,1,0,0,0,1,...,1,0,0,1,0,0,0,-0.465369,-0.761147,-0.636722
2326,0,0,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,-0.302393,-1.263077,-0.683730
2921,0,0,1,0,1,1,1,1,1,1,...,1,0,0,0,0,0,1,1.490341,1.701970,2.602807
5391,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0.023559,0.186206,0.054178
1413,1,0,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,1.408853,0.822761,1.804368
1781,0,0,0,0,1,0,1,1,1,0,...,0,0,0,0,1,0,0,0.227279,0.646586,0.432893
1141,0,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,-0.017185,0.355732,0.018265
4004,0,0,1,1,1,0,0,0,0,0,...,0,1,0,1,0,0,0,-0.383881,-1.500746,-0.808145
3841,0,0,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,0,-1.239504,-0.518491,-0.961612


In [169]:
test_X.reset_index(inplace=True)

In [170]:
export_df = pd.concat([test_X, pd.DataFrame(pred_y)], axis=1)

In [171]:
export_df

,index,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges,0
0,3464,1,0,1,1,1,1,0,0,0,...,0,1,0,0,0,0,-0.995040,-0.448686,-0.825771,0
1,354,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,-0.465369,-0.761147,-0.636722,0
2,2326,0,0,1,0,1,0,0,0,0,...,1,0,0,1,0,0,-0.302393,-1.263077,-0.683730,0
3,2921,0,0,1,0,1,1,1,1,1,...,0,0,0,0,0,1,1.490341,1.701970,2.602807,0
4,5391,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0.023559,0.186206,0.054178,0
5,1413,1,0,1,1,1,0,1,0,0,...,0,0,0,0,0,1,1.408853,0.822761,1.804368,0
6,1781,0,0,0,0,1,0,1,1,1,...,0,0,0,1,0,0,0.227279,0.646586,0.432893,0
7,1141,0,0,1,0,1,1,0,0,0,...,0,0,0,1,0,0,-0.017185,0.355732,0.018265,0
8,4004,0,0,1,1,1,0,0,0,0,...,1,0,1,0,0,0,-0.383881,-1.500746,-0.808145,0
9,3841,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,-1.239504,-0.518491,-0.961612,0


In [177]:
accuracy = accuracy_score(test_Y,pred_y)
accuracy

0.7952218430034129

In [180]:
# Looking at the Customers for which Churn is predicted to be Yes
export_df[export_df[0]==1]

,index,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges,0
30,3869,1,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,-0.424625,1.191729,0.058523,1
34,5263,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,-1.076528,0.841043,-0.777373,1
47,5774,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,-1.280248,0.814450,-0.967965,1
54,2575,0,1,0,1,1,0,0,0,1,...,0,0,0,1,0,0,-0.098673,1.213336,0.304949,1
56,6313,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,-1.158016,0.472074,-0.873618,1
61,6368,1,0,0,0,0,0,1,0,1,...,1,1,0,0,0,0,-1.280248,-0.669735,-0.987665,1
65,863,1,0,1,0,1,0,0,1,1,...,1,1,0,0,0,0,-1.158016,0.525259,-0.866559,1
74,238,0,1,0,0,1,0,1,0,0,...,0,1,0,0,0,0,-0.872808,1.003921,-0.513101,1
78,4751,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,-1.280248,0.470412,-0.972532,1
87,3971,1,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,-1.198760,0.860987,-0.902515,1
